In [43]:
import geopandas as gpd
import country_converter as coco
import pandas as pd
import json
import folium
from shapely.geometry import *
import tempfile


In [2]:
#File path
# shapefile = '/content/drive/My Drive/data/World happiness/Country JSON/ne_50m_admin_0_countries.shp'
shapefile = 'ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp'

#Read shapefile into pandas dataframe
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
#Rename columns
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head(10)

,country,country_code,geometry
0,Indonesia,IDN,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
1,Malaysia,MYS,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4..."
2,Chile,CHL,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611..."
3,Bolivia,BOL,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5..."
4,Peru,PER,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832..."
5,Argentina,ARG,"MULTIPOLYGON (((-67.28475 -23.83414, -67.25133..."
6,Dhekelia Sovereign Base Area,ESB,"POLYGON ((33.78094 34.97635, 33.76043 34.97968..."
7,Cyprus,CYP,"MULTIPOLYGON (((33.78183 34.97622, 33.78094 34..."
8,India,IND,"MULTIPOLYGON (((77.80035 35.49541, 77.81533 35..."
9,China,CHN,"MULTIPOLYGON (((78.91595 33.38766, 78.82426 33..."


In [3]:
# check that iso codes match

In [4]:
gdf.loc[gdf['country'] == "Finland"]

,country,country_code,geometry
55,Finland,FIN,"MULTIPOLYGON (((28.95408 69.02726, 28.83346 68..."


### merge csvs to get data with iso codes

In [5]:
isocode_df= pd.read_csv('../CSV_Files/countries_codes_and_coordinates.csv')
isocode_df

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,"""AF""","""AFG""","""4""","""33""","""65"""
1,Albania,"""AL""","""ALB""","""8""","""41""","""20"""
2,Algeria,"""DZ""","""DZA""","""12""","""28""","""3"""
3,American Samoa,"""AS""","""ASM""","""16""","""-14.3333""","""-170"""
4,Andorra,"""AD""","""AND""","""20""","""42.5""","""1.6"""
...,...,...,...,...,...,...
251,Wallis and Futuna,"""WF""","""WLF""","""876""","""-13.3""","""-176.2"""
252,Western Sahara,"""EH""","""ESH""","""732""","""24.5""","""-13"""
253,Yemen,"""YE""","""YEM""","""887""","""15""","""48"""
254,Zambia,"""ZM""","""ZMB""","""894""","""-15""","""30"""


In [6]:
isocode_df= isocode_df[["Country", "Alpha-3 code"]]
isocode_df= isocode_df.rename(columns={"Country": "Country name","Alpha-3 code": "country_code"})

isocode_df

,Country name,country_code
0,Afghanistan,"""AFG"""
1,Albania,"""ALB"""
2,Algeria,"""DZA"""
3,American Samoa,"""ASM"""
4,Andorra,"""AND"""
...,...,...
251,Wallis and Futuna,"""WLF"""
252,Western Sahara,"""ESH"""
253,Yemen,"""YEM"""
254,Zambia,"""ZMB"""


In [7]:
isocode_df.country_code = isocode_df.country_code.replace({'"':''}, regex=True)
isocode_df

,Country name,country_code
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND
...,...,...
251,Wallis and Futuna,WLF
252,Western Sahara,ESH
253,Yemen,YEM
254,Zambia,ZMB


In [8]:
#csv 1

happiness_df= pd.read_csv('../CSV_Files/world-happiness-report-2021.csv')
happiness_df

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.000,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.700,0.946,0.030,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.400,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.000,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.400,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Lesotho,Sub-Saharan Africa,3.512,0.120,3.748,3.276,7.926,0.787,48.700,0.715,-0.131,0.915,2.43,0.451,0.731,0.007,0.405,0.103,0.015,1.800
145,Botswana,Sub-Saharan Africa,3.467,0.074,3.611,3.322,9.782,0.784,59.269,0.824,-0.246,0.801,2.43,1.099,0.724,0.340,0.539,0.027,0.088,0.648
146,Rwanda,Sub-Saharan Africa,3.415,0.068,3.548,3.282,7.676,0.552,61.400,0.897,0.061,0.167,2.43,0.364,0.202,0.407,0.627,0.227,0.493,1.095
147,Zimbabwe,Sub-Saharan Africa,3.145,0.058,3.259,3.030,7.943,0.750,56.201,0.677,-0.047,0.821,2.43,0.457,0.649,0.243,0.359,0.157,0.075,1.205


In [9]:
final_happiness_df= happiness_df.merge(isocode_df)
final_happiness_df.head()

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,...,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual,country_code
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,...,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253,FIN
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.7,0.946,...,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868,DNK
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.4,0.919,...,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839,CHE
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.0,0.955,...,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967,ISL
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.4,0.913,...,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798,NLD


In [10]:
#Save df to csv
final_happiness_df.to_csv('../CSV_Files/final_happiness_2021.csv',index=False)

In [11]:
#see if order of columns is giving merge issue ????

# test12= final_happiness_df[['country_code', 'Country name',	'Regional indicator',	'Ladder score',	'Standard error of ladder score',	'upperwhisker',	'lowerwhisker',	'Logged GDP per capita',	'Social support',	'Healthy life expectancy',	'Freedom to make life choices', 'Generosity', 'Perceptions of corruption', 'Ladder score in Dystopia', 'Explained by: Log GDP per capita',	'Explained by: Social support',	'Explained by: Healthy life expectancy',	'Explained by: Freedom to make life choices',	'Explained by: Generosity',	'Explained by: Perceptions of corruption',	'Dystopia + residual']]
# test12

In [22]:
final_happiness_df.head(2)

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,...,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual,country_code
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,...,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253,FIN
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.7,0.946,...,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868,DNK


In [23]:
gdf.head(2)

,country,country_code,geometry
0,Indonesia,IDN,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
1,Malaysia,MYS,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4..."


In [27]:
gdf["country_code"][1]

'MYS'

In [30]:
final_happiness_df["country_code"][20]

'FRA'

In [33]:
final_happiness_df.country_code = final_happiness_df.country_code.replace({' ':''}, regex=True)
final_happiness_df.head(2)

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,...,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual,country_code
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,...,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253,FIN
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.7,0.946,...,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868,DNK


### ************  why does the first way not work??

In [32]:
final_merged_df= pd.merge(gdf, final_happiness_df, how= "inner", on= "country_code")
final_merged_df

,country,country_code,geometry,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,...,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Indonesia,IDN,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4...",Indonesia,Southeast Asia,5.345,0.056,5.454,5.235,9.365,...,0.542,0.867,2.43,0.954,0.786,0.433,0.598,0.541,0.046,1.987
1,Malaysia,MYS,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4...",Malaysia,Southeast Asia,5.384,0.049,5.480,5.289,10.238,...,0.125,0.839,2.43,1.259,0.797,0.587,0.624,0.270,0.064,1.784
2,Chile,CHL,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611...",Chile,Latin America and Caribbean,6.172,0.046,6.262,6.081,10.071,...,-0.044,0.830,2.43,1.200,0.946,0.678,0.438,0.159,0.070,2.682
3,Bolivia,BOL,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5...",Bolivia,Latin America and Caribbean,5.716,0.053,5.819,5.613,9.046,...,-0.077,0.839,2.43,0.842,0.782,0.486,0.600,0.138,0.064,2.805
4,Peru,PER,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832...",Peru,Latin America and Caribbean,5.840,0.075,5.988,5.692,9.458,...,-0.154,0.891,2.43,0.986,0.833,0.623,0.536,0.087,0.031,2.744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Comoros,COM,"MULTIPOLYGON (((43.78875 -12.30804, 43.79705 -...",Comoros,Sub-Saharan Africa,4.289,0.084,4.454,4.123,8.031,...,0.082,0.781,2.43,0.488,0.367,0.279,0.202,0.241,0.101,2.610
134,Malta,MLT,"MULTIPOLYGON (((14.54802 35.89004, 14.56316 35...",Malta,Western Europe,6.602,0.044,6.688,6.516,10.674,...,0.133,0.653,2.43,1.411,1.055,0.747,0.664,0.275,0.183,2.268
135,Singapore,SGP,"POLYGON ((103.96079 1.39110, 103.98569 1.38544...",Singapore,Southeast Asia,6.377,0.043,6.460,6.293,11.488,...,-0.018,0.082,2.43,1.695,1.019,0.897,0.664,0.176,0.547,1.379
136,Maldives,MDV,"MULTIPOLYGON (((73.16310 -0.68182, 73.17254 -0...",Maldives,South Asia,5.198,0.072,5.339,5.057,9.826,...,0.024,0.825,2.43,1.115,1.015,0.697,0.575,0.204,0.073,1.520


In [34]:
# merged_df=gdf.merge(final_happiness_df,left_on='country_code',right_on='country_code')
# # merged_df=pd.merge(gdf, final_happiness_df, how='outer', on='country_code')


# merged_df


In [35]:
# merged_df=gdf.merge(final_happiness_df,left_on='country',right_on='Country name')
# # merged_df=pd.merge(gdf, final_happiness_df, how='outer', on='country_code')


# merged_df.head(3)


In [36]:
# merged_df= merged_df.drop(['Country name', 'country_code_y'], axis=1)

In [37]:
merged_json = json.loads(final_merged_df.to_json())
# json_data = json.dumps(merged_json)

## save geojson file to use for choropleth

In [48]:
# json_data

In [45]:
# def write_json(self, features):
#    # feature is a shapely geometry type
#    geom_in_geojson = json_data.Feature(geometry=features, properties={})
#    tmp_file = tempfile.mkstemp(suffix='.geojson')
#    with open(tmp_file[1], 'w') as outfile:
#       geojson.dump(geom_in_geojson, outfile)
#    return tmp_file[1]

In [51]:
with open('choro_data.geojson', 'w', encoding='utf-8') as f:
    json.dump(merged_json, f, ensure_ascii=False, indent=4)